[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/IST-DASLab/sparsegpt/blob/master/demo.ipynb)

Install dependencies

In [1]:
!pip install -q datasets
!pip install -q transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Clone repository

In [2]:
!git clone https://github.com/IST-DASLab/sparsegpt

Cloning into 'sparsegpt'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 46 (delta 22), reused 13 (delta 10), pack-reused 11 (from 1)
Receiving objects: 100% (46/46), 26.80 KiB | 13.40 MiB/s, done.
Resolving deltas: 100% (22/22), done.


### Pruning example
---

Below we will show an example of SparseGPT applied to OPT model.

In [3]:
%cd sparsegpt

/content/sparsegpt


Crerate directory to store prune model(s)

In [4]:
!mkdir -p sparse_opt

We will use `opt.py` script to prune the model.
Select one of the following OPT versions to fit into colab (with `bitsandbytes` one should be able to use larger 6.7b and 13b models):
* facebook/opt-125m
* facebook/opt-350m
* facebook/opt-1.3b

To prune the model select dataset for calibration (`c4`, `ptb` or `wikitext`). The SparseGPT paper uses `c4` by default.

One can prune model to uniform sparsity with SparseGPT either with unstructured pruning or semistructured `N:M` pattern.

To apply unstructured pruning specify `--sparsity` - floating point number in `[0, 1]`.

For semitstructured specify `--prunen` and `--prunem` arguments - integer numbers.

To apply magnitude pruning instead of SparseGPT select `--gmp` option.

To apply quantization on top of sparsity specify `--wbits`.

In the example below we prune `facebook/opt-125m` to 0.5 unstructured sparsity via SparseGPT. Try different options.


In [5]:
!python opt.py facebook/opt-125m c4 --sparsity 0.5 --save sparse_opt/opt-125m

2024-12-13 17:38:05.154278: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-13 17:38:05.173659: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-13 17:38:05.179436: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 17:38:05.193330: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-13 17:38:06.461245: W tensorflow/comp

Code above prints perplexity on `wikitext2`, `ptb` and `c4` benchmarks in the end.

### Compare generations
---

Let us compare generations produced by the dense and sparse model

In [6]:
from transformers import AutoTokenizer, OPTForCausalLM

In [7]:
device = 'cuda'

In [8]:
# load dense model
model_dn = OPTForCausalLM.from_pretrained('facebook/opt-125m', torch_dtype='auto').to(device)
# load sparse model
model_sp = OPTForCausalLM.from_pretrained('sparse_opt/opt-125m', torch_dtype='auto').to(device)
# init tokenizer
tokenizer = AutoTokenizer.from_pretrained('facebook/opt-125m')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: sparse_opt/opt-125m is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
input_text = "It takes a great deal of bravery"

In [ ]:
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

Completion by dense model:

In [ ]:
output_ids = model_dn.generate(input_ids)

In [ ]:
print(tokenizer.decode(output_ids[0].cpu(), skip_special_tokens=True))

Completion by sparse model:

In [ ]:
output_ids = model_sp.generate(input_ids)

In [ ]:
print(tokenizer.decode(output_ids[0].cpu(), skip_special_tokens=True))